### Load required packages

In [1]:
import pandas as pd
import numpy as np

### Load data

In [3]:
data = pd.read_csv('https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv')
data

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1
...,...,...,...,...,...,...,...,...,...
1457,referral,manufacturing,1,NaN,self_employed,north_america,4,0.53,1
1458,referral,technology,3,65259.0,student,europe,2,0.24,1
1459,paid_ads,technology,1,45688.0,student,north_america,3,0.02,1
1460,referral,NaN,5,71016.0,self_employed,north_america,0,0.25,1


In [7]:
data.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [8]:
data.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [10]:
data.fillna({'lead_source': 'NA', 'industry': 'NA', 'annual_income': 0, 'employment_status': 'NA', 'location': 'NA'}, inplace = True)
data.isnull().sum()

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

In [14]:
numerical_variables = ['number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score']
categorical_variables = ['lead_source', 'industry', 'employment_status', 'location']
target_variable = ['converted']

### 1. What is the most frequent observation (mode) for the column industry?

In [11]:
data.industry.mode()

0    retail
Name: industry, dtype: object

### 2. What are the two features that have the biggest correlation?

Create the correlation matrix for the numerical features of your dataset. In a correlation matrix, you compute the correlation coefficient between every pair of features.

In [15]:
data[numerical_variables].corr()

,number_of_courses_viewed,annual_income,interaction_count,lead_score
number_of_courses_viewed,1.000000,0.009770,-0.023565,-0.004879
annual_income,0.009770,1.000000,0.027036,0.015610
interaction_count,-0.023565,0.027036,1.000000,0.009888
lead_score,-0.004879,0.015610,0.009888,1.000000


### Split data using scikit-learn

- Split your data in train/val/test sets with 60%/20%/20% distribution.
- Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.
- Make sure that the target value y is not in your dataframe.

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
data_train_full, data_test = train_test_split(data, test_size = 0.2, train_size = 0.8, random_state = 42)
data_train_full.shape, data_test.shape

((1169, 9), (293, 9))

In [18]:
data_train, data_val = train_test_split(data_train_full, test_size = 0.25, random_state = 42)
data_train.shape, data_val.shape

((876, 9), (293, 9))

In [20]:
# reset indices
data_train_full.reset_index(drop = True, inplace = True)
data_train.reset_index(drop = True, inplace = True)
data_val.reset_index(drop = True, inplace = True)
data_test.reset_index(drop = True, inplace = True)

In [22]:
# get target variables
y_train_full = data_train_full.converted.values
y_train = data_train.converted.values
y_val = data_val.converted.values
y_test = data_test.converted.values

In [23]:
# delete y from dataset -> this avoids accidentally adding variable to model
del data_train['converted']
del data_val['converted']
del data_test['converted']

### 3. Which of these variables has the biggest mutual information score?

- Calculate the mutual information score between y and other categorical variables in the dataset. Use the training set only.
- Round the scores to 2 decimals using round(score, 2).

In [24]:
from sklearn.metrics import mutual_info_score

In [30]:
def calculate_mutual_info_score(series):
    return round(mutual_info_score(y_train, series), 5)

In [31]:
df_mutual_info = data_train[categorical_variables].apply(calculate_mutual_info_score)
df_mutual_info = df_mutual_info.sort_values(ascending=False).to_frame(name='MI')
df_mutual_info

,MI
lead_source,0.03540
employment_status,0.01294
industry,0.01157
location,0.00446


### 4. What accuracy did you get?

- Now let's train a logistic regression.
- Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.
- Fit the model on the training dataset.
    - To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
    - <code> model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42) </code>
- Calculate the accuracy on the validation dataset and round it to 2 decimal digits.

In [34]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

In [35]:
train_dicts = data_train[categorical_variables + numerical_variables].to_dict(orient='records')
dv_train = DictVectorizer(sparse=False)
X_train = dv_train.fit_transform(train_dicts)
X_train

array([[5.8472e+04, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [7.1738e+04, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        3.0000e+00],
       [8.1973e+04, 0.0000e+00, 1.0000e+00, ..., 1.0000e+00, 0.0000e+00,
        3.0000e+00],
       ...,
       [8.9042e+04, 0.0000e+00, 1.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        3.0000e+00],
       [0.0000e+00, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        1.0000e+00],
       [5.0259e+04, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        4.0000e+00]], shape=(876, 31))

In [36]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'liblinear'
,max_iter,1000
,multi_class,'deprecated'


In [51]:
y_train_pred = model.predict(X_train)
y_train_pred_prod = model.predict_proba(X_train)[:, 1]
df_train_pred = pd.DataFrame()
df_train_pred['probability'] = y_train_pred_prod
df_train_pred['prediction'] = y_train_pred
df_train_pred['actual'] = y_train
df_train_pred['correct'] = y_train == y_train_pred
original_accuracy = df_train_pred['correct'].mean()
print(f"Accuracy on test data: {original_accuracy}")
df_train_pred

Accuracy on test data: 0.7385844748858448


,probability,prediction,actual,correct
0,0.579143,1,0,False
1,0.872835,1,1,True
2,0.588161,1,1,True
3,0.512312,1,1,True
4,0.636665,1,1,True
...,...,...,...,...
871,0.705462,1,1,True
872,0.597729,1,0,False
873,0.747342,1,1,True
874,0.669784,1,0,False


- Let's find the least useful feature using the feature elimination technique.
- Train a model using the same features and parameters as in Q4 (without rounding).
- Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
- For each feature, calculate the difference between the original accuracy and the accuracy without the feature.

### 5. Which of following feature has the smallest difference?

- 'industry'
- 'employment_status'
- 'lead_score'

In [52]:
all_features = set(categorical_variables + numerical_variables)
feature_model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
for feature in all_features:
    model_features = list(all_features - set([feature]))
    feature_train_dicts = data_train[model_features].to_dict(orient='records')
    dv_feature_train = DictVectorizer(sparse=False)
    X_feature_train = dv_feature_train.fit_transform(feature_train_dicts)
    
    feature_model.fit(X_feature_train, y_train)
    y_feature_train_pred = feature_model.predict(X_feature_train)
    feature_accuracy = (y_train == y_feature_train_pred).mean()
    print(f"Accuracy without feature {feature} is {feature_accuracy} and difference from original is {feature_accuracy - original_accuracy}")

Accuracy without feature number_of_courses_viewed is 0.6232876712328768 and difference from original is -0.11529680365296802
Accuracy without feature lead_source is 0.7420091324200914 and difference from original is 0.003424657534246589
Accuracy without feature location is 0.7420091324200914 and difference from original is 0.003424657534246589
Accuracy without feature employment_status is 0.7351598173515982 and difference from original is -0.003424657534246589
Accuracy without feature lead_score is 0.7420091324200914 and difference from original is 0.003424657534246589
Accuracy without feature annual_income is 0.8710045662100456 and difference from original is 0.13242009132420085
Accuracy without feature industry is 0.7408675799086758 and difference from original is 0.0022831050228310223
Accuracy without feature interaction_count is 0.6255707762557078 and difference from original is -0.113013698630137


- Now let's train a regularized logistic regression.
- Let's try the following values of the parameter C: [0.01, 0.1, 1, 10, 100].
- Train models using all the features as in Q4.
- Calculate the accuracy on the validation dataset and round it to 3 decimal digits.

### 6. Which of these <code>C</code> leads to the best accuracy on the validation set?

In [53]:
reg_cs = [0.01, 0.1, 1, 10, 100]
for c in reg_cs:    
    model_c = LogisticRegression(solver='liblinear', C=c, max_iter=1000, random_state=42)
    model_c.fit(X_train, y_train)
    y_train_pred_c = model_c.predict(X_train)
    accuracy_c = (y_train == y_train_pred_c).mean()
    print(f"Accuracy for {c} is {accuracy_c}")

Accuracy for 0.01 is 0.7465753424657534
Accuracy for 0.1 is 0.7397260273972602
Accuracy for 1 is 0.7385844748858448
Accuracy for 10 is 0.7385844748858448
Accuracy for 100 is 0.7385844748858448
